  Préfixo:
  
  Tabelas
  * tb_

  View
  * vw_

  Stored Procedure
  * sp_

  Function
  * fn_

  Trigger
  * tr_

  Índice
  * ix_

  Chave Primaria
  * pk_

  Chave Estrangeira
  * fk_


In [ ]:
output_file = "001_clean_table_bikestores_ddl_ADMIN.sql"

sql_script = """
-- 1. Limpeza inicial (opcional - rode com cuidado para dropar tabelas existentes)
-- Usamos blocos PL/SQL para ignorar erros se objetos não existirem

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE sales_tb_order_items CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE sales_tb_orders CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE sales_tb_customers CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE sales_tb_staffs CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE sales_tb_stores CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE production_tb_stocks CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE production_tb_products CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE production_tb_categories CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/

BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE production_tb_brands CASCADE CONSTRAINTS';
EXCEPTION WHEN OTHERS THEN NULL;
END;
/


"""

with open(output_file, "w", encoding="utf-8") as arquivo:
    arquivo.write(sql_script)

In [ ]:
output_file = "002_create_table_bikestores_ddl_bikestore.sql"

sql_script = """
-- 2. Criação das tabelas ajustadas para Oracle (use NUMBER para integers/decimals, NUMBER(1) para boolean)
-- Rode como SYSTEM ou em um usuário dedicado (ex: bikestore)

-- production
CREATE TABLE production_tb_brands (
    brand_id    NUMBER PRIMARY KEY,
    brand_name  VARCHAR2(100) NOT NULL
);

CREATE TABLE production_tb_categories (
    category_id    NUMBER PRIMARY KEY,
    category_name  VARCHAR2(100) NOT NULL
);

CREATE TABLE production_tb_products (
    product_id     NUMBER PRIMARY KEY,
    product_name   VARCHAR2(255) NOT NULL,
    brand_id       NUMBER NOT NULL,
    category_id    NUMBER NOT NULL,
    product_model_year     NUMBER(4)     NOT NULL,
    product_list_price     NUMBER(10,2)  NOT NULL,
    CONSTRAINT fk_products_brand    FOREIGN KEY (brand_id)    REFERENCES production_tb_brands(brand_id),
    CONSTRAINT fk_products_category FOREIGN KEY (category_id) REFERENCES production_tb_categories(category_id)
);


-- sales
CREATE TABLE sales_tb_stores (
    store_id    NUMBER PRIMARY KEY,
    store_name  VARCHAR2(100) NOT NULL,
    store_phone       VARCHAR2(25),
    store_email       VARCHAR2(100),
    store_street      VARCHAR2(100),
    store_city        VARCHAR2(50),
    store_state       CHAR(2),
    store_zip_code    VARCHAR2(10)
);

CREATE TABLE production_tb_stocks (
    store_id       NUMBER NOT NULL,
    product_id     NUMBER NOT NULL,
    stock_quantity       NUMBER NOT NULL,
    CONSTRAINT pk_tb_stocks      PRIMARY KEY (store_id, product_id),
    CONSTRAINT fk_stocks_store   FOREIGN KEY (store_id)   REFERENCES sales_tb_stores(store_id),
    CONSTRAINT fk_stocks_product FOREIGN KEY (product_id) REFERENCES production_tb_products(product_id)
);

CREATE TABLE sales_tb_staffs (
    staff_id       NUMBER PRIMARY KEY,
    staff_first_name     VARCHAR2(50)  NOT NULL,
    staff_last_name      VARCHAR2(50)  NOT NULL,
    staff_email          VARCHAR2(100) UNIQUE NOT NULL,
    staff_phone          VARCHAR2(25),
    staff_active         NUMBER(1)     NOT NULL,
    store_id       NUMBER        NOT NULL,
    manager_id     NUMBER,
    CONSTRAINT fk_staffs_store   FOREIGN KEY (store_id)   REFERENCES sales_tb_stores(store_id),
    CONSTRAINT fk_staffs_manager FOREIGN KEY (manager_id) REFERENCES sales_tb_staffs(staff_id)
);

CREATE TABLE sales_tb_customers (
    customer_id    NUMBER PRIMARY KEY,
    customer_first_name     VARCHAR2(50)  NOT NULL,
    customer_last_name      VARCHAR2(50)  NOT NULL,
    customer_phone          VARCHAR2(25),
    customer_email          VARCHAR2(100) UNIQUE NOT NULL,
    customer_street         VARCHAR2(100),
    customer_city           VARCHAR2(50),
    customer_state          CHAR(2),
    customer_zip_code       VARCHAR2(10)
);

CREATE TABLE sales_tb_orders (
    order_id        NUMBER PRIMARY KEY,
    customer_id     NUMBER NOT NULL,
    order_status    NUMBER(2) NOT NULL,
    order_date      DATE NOT NULL,
    order_required_date   DATE NOT NULL,
    order_shipped_date    DATE,
    store_id        NUMBER NOT NULL,
    staff_id        NUMBER NOT NULL,
    CONSTRAINT fk_orders_customer FOREIGN KEY (customer_id) REFERENCES sales_tb_customers(customer_id),
    CONSTRAINT fk_orders_store    FOREIGN KEY (store_id)    REFERENCES sales_tb_stores(store_id),
    CONSTRAINT fk_orders_staff    FOREIGN KEY (staff_id)    REFERENCES sales_tb_staffs(staff_id)
);

CREATE TABLE sales_tb_order_items (
    order_id       NUMBER       NOT NULL,
    item_id        NUMBER       NOT NULL,
    product_id     NUMBER       NOT NULL,
    order_item_quantity       NUMBER       NOT NULL,
    order_item_list_price     NUMBER(10,2) NOT NULL,
    order_item_discount       NUMBER(4,2)  NOT NULL,
    CONSTRAINT pk_order_items       PRIMARY KEY (order_id, item_id),
    CONSTRAINT fk_order_items_order FOREIGN KEY (order_id)   REFERENCES sales_tb_orders(order_id),
    CONSTRAINT fk_order_items_product FOREIGN KEY (product_id) REFERENCES production_tb_products(product_id)
);
"""

with open(output_file, "w", encoding="utf-8") as arquivo:
    arquivo.write(sql_script)

In [ ]:
import pandas as pd

arquivo_brands = pd.read_csv("brands.csv")
arquivo_brands

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [ ]:
linhas = []

for _, linha in arquivo_brands.iterrows():
  valores_linha = []
  for valor in linha:
    if pd.isna(valor):
      valores_linha.append("NULL")
    elif isinstance(valor, str):
      valor_escapa = valor.replace("'", "''")
      valores_linha.append(f"'{valor_escapa}'")
    else:
      valores_linha.append(str(valor))
  linha_formatada = '(' + ','.join(valores_linha) + ')'
  linhas.append(linha_formatada)

In [ ]:
output_file = '001_insert_table_production_tb_brands_dml_bikestore.sql'

inserir = 'INTO'
tabela = 'production_tb_brands'
colunas = '(' + ', '.join(arquivo_brands.columns) + ')'
inserir_valores = []
for i in linhas:
  inserir_valores.append(f'{inserir} {tabela} {colunas} VALUES {i}')
inserir_valores = '\n   '.join(inserir_valores)

sql_script_insert = f'''INSERT ALL
   {inserir_valores}
SELECT 1 FROM dual;

COMMIT;
  '''
with open(output_file, "w", encoding="utf-8") as arquivo:
    arquivo.write(sql_script_insert)

In [1]:
import pandas as pd
import os
from pathlib import Path


pasta_csv = "/content/arquivos_csv"
pasta_saida = "./sql_output"
categoria = "production"

os.makedirs(pasta_saida, exist_ok=True)

arquivos_csv = [f for f in os.listdir(pasta_csv) if f.endswith('.csv')]

print(f"Encontrados {len(arquivos_csv)} arquivos CSV")

for idx, arquivo_nome in enumerate(sorted(arquivos_csv), start=1):
    try:
        print(f"\nProcessando {idx}/{len(arquivos_csv)}: {arquivo_nome}")

        caminho_completo = os.path.join(pasta_csv, arquivo_nome)
        arquivo = pd.read_csv(caminho_completo)

        nome_base = arquivo_nome.replace('.csv', '')

        linhas = []
        for _, linha in arquivo.iterrows():
            valores_linha = []
            for valor in linha:
                if pd.isna(valor):
                    valores_linha.append("NULL")
                elif isinstance(valor, str):
                    valor_escapa = valor.replace("'", "''")
                    valores_linha.append(f"'{valor_escapa}'")
                else:
                    valores_linha.append(str(valor))
            linha_formatada = '(' + ','.join(valores_linha) + ')'
            linhas.append(linha_formatada)

        numeracao = str(idx).zfill(3)
        output_file = os.path.join(pasta_saida,
                                   f'{numeracao}_insert_table_{categoria}_tb_{nome_base}_dml_bikestore.sql')

        # Montar o script SQL (usando UNION ALL - mais eficiente para Oracle 21c)
        tabela = f'{categoria}_tb_{nome_base}'
        colunas = '(' + ', '.join(arquivo.columns) + ')'

        select_valores = []
        for i, valores in enumerate(linhas):
            valores_sem_parenteses = valores[1:-1]
            if i == 0:
                select_valores.append(f'SELECT {valores_sem_parenteses} FROM dual')
            else:
                select_valores.append(f'SELECT {valores_sem_parenteses} FROM dual')

        sql_script_insert = f'''INSERT INTO {tabela} {colunas}
{' UNION ALL\n'.join(select_valores)};

COMMIT;
'''

        with open(output_file, "w", encoding="utf-8") as arquivo_sql:
            arquivo_sql.write(sql_script_insert)

        print(f"✓ Gerado: {output_file} ({len(linhas)} registros)")

    except Exception as e:
        print(f"✗ Erro ao processar {arquivo_nome}: {str(e)}")

print(f"\n{'='*50}")
print(f"Processamento concluído!")
print(f"Arquivos SQL salvos em: {pasta_saida}")

Encontrados 4 arquivos CSV

Processando 1/4: brands.csv
✓ Gerado: ./sql_output/001_insert_table_production_tb_brands_dml_bikestore.sql (9 registros)

Processando 2/4: categories.csv
✓ Gerado: ./sql_output/002_insert_table_production_tb_categories_dml_bikestore.sql (7 registros)

Processando 3/4: products.csv
✓ Gerado: ./sql_output/003_insert_table_production_tb_products_dml_bikestore.sql (321 registros)

Processando 4/4: stocks.csv
✓ Gerado: ./sql_output/004_insert_table_production_tb_stocks_dml_bikestore.sql (939 registros)

Processamento concluído!
Arquivos SQL salvos em: ./sql_output


In [2]:
from google.colab import files
import shutil

shutil.make_archive("production", "zip", "/content/sql_output")
files.download("production.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>